In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
import cartopy.feature as cfeature
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d 
from scipy import ndimage
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras    
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.models import load_model
from sklearn.cluster import KMeans, MiniBatchKMeans
import joblib
import plot_functions
import importlib 
importlib.reload(plot_functions)
from plot_functions import *
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
# Visualize the result

total cores: 256


In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import socket
hostname = socket.gethostname()
if "nird" in hostname:
    tf.config.threading.set_inter_op_parallelism_threads(8)
    data_loc = "/nird/projects/NS9600K/fslippe/data/"
    folder = "/nird/projects/NS9600K/data/modis/cao/MOD02/2020/ /nird/projects/NS9600K/data/modis/cao/MOD02/2021/ /nird/projects/NS9600K/data/modis/cao/MOD02/2023/"
if "mimi" in hostname:
    data_loc = "/uio/hume/student-u37/fslippe/data/"
    folder = "/scratch/fslippe/modis/MOD02/2019/ /scratch/fslippe/modis/MOD02/2020/ /scratch/fslippe/modis/MOD02/2021/ /scratch/fslippe/modis/MOD02/2022/ /scratch/fslippe/modis/MOD02/2023/ /scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/ /scratch/fslippe/modis/MOD02/may-nov_2021/ /scratch/fslippe/modis/MOD02/cao_test_data/"


bands=[29]
max_vals = np.load("%smodels/max_vals_dnb_l95_z50_ps128_(29)_cao_months_202012-202111.npy" %data_loc)
from autoencoder import SobelFilterLayer, SimpleAutoencoder
print(len(bands))

#encoder = load_model("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_encoder")
encoder = load_model("%smodels/encoder_scheduler_250k_dnb_l90_z50_fcao_months_202012_(29)_202111-64" %data_loc)

1 Physical GPUs, 1 Logical GPUs
1


In [5]:
start = "20190101"
end = "20190430"
dates = generate_date_list(start, end)
start = "20191001"
end = "20191231"
dates.extend(generate_date_list(start, end))

x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao = extract_1km_data(folder,
                                                         bands=bands,
                                                         date_list=dates,
                                                         return_lon_lat=True,
                                                         data_loc=data_loc,
                                                         data_type="npy",
                                                         combine_pics=True)
x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao = zip(*[(xi, date, mask, lon_lat, mod_min) for xi, date, mask, lon_lat, mod_min in zip(x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao) if (xi.shape[0] > patch_size) and (xi.shape[1] > patch_size)])
x_cao = list(x_cao)
dates_cao = list(dates_cao)
len(x_cao)

['/scratch/fslippe/modis/MOD02/2019/', '/scratch/fslippe/modis/MOD02/2020/', '/scratch/fslippe/modis/MOD02/2021/', '/scratch/fslippe/modis/MOD02/2022/', '/scratch/fslippe/modis/MOD02/2023/', '/scratch/fslippe/modis/MOD02/daytime_1km/', '/scratch/fslippe/modis/MOD02/boundary_1km/', '/scratch/fslippe/modis/MOD02/night_1km/', '/scratch/fslippe/modis/MOD02/may-nov_2021/', '/scratch/fslippe/modis/MOD02/cao_test_data/']
['2019001', '2019002', '2019003', '2019004', '2019005', '2019006', '2019007', '2019008', '2019009', '2019010', '2019011', '2019012', '2019013', '2019014', '2019015', '2019016', '2019017', '2019018', '2019019', '2019020', '2019021', '2019022', '2019023', '2019024', '2019025', '2019026', '2019027', '2019028', '2019029', '2019030', '2019031', '2019032', '2019033', '2019034', '2019035', '2019036', '2019037', '2019038', '2019039', '2019040', '2019041', '2019042', '2019043', '2019044', '2019045', '2019046', '2019047', '2019048', '2019049', '2019050', '2019051', '2019052', '2019053'

100%|██████████| 212/212 [04:20<00:00,  1.23s/it]


2529
2529


2171

In [15]:
dates_block = np.load("/uio/hume/student-u37/fslippe/data/dates_for_labeling/day_filtered/dates_block.npy")
times_block = np.load("/uio/hume/student-u37/fslippe/data/dates_for_labeling/day_filtered/times_block.npy")


x, dates, masks, lon_lats, mod_min = zip(*[(xi, date, mask, lon_lat, mod_min) for xi, date, mask, lon_lat, mod_min in zip(x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao) if not (date, mod_min) in zip(dates_block, times_block)])


In [27]:
import importlib
import autoencoder
importlib.reload(autoencoder)
from autoencoder import * 
all_patches = []
i=0
tot = len(x)
patch_size = 64
strides = [1, patch_size, patch_size, 1]
lon_lat_min_max = [-35, 45, 60, 82]
autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)


for (image, mask, lon_lat) in zip(x, masks, lon_lats):
    print(f"{i+1}/{tot}", end="\r")
    patches, idx, n_patches, lon, lat = autoencoder.extract_patches(image,
                                                                    mask,
                                                                    mask_threshold=0.95,
                                                                    lon_lat=lon_lat,
                                                                    extract_lon_lat=True,
                                                                    strides=strides,
                                                                    lon_lat_min_max=lon_lat_min_max) 
    
    all_patches.append(patches)
    i+=1

patches = np.concatenate(all_patches, axis=0)

: 

In [ ]:
len(patches)

143065

In [ ]:
# TRAIN TEST SPLIT
patches, val_data = train_test_split(patches, test_size=0.15, random_state=42, shuffle=True)

In [ ]:
patches.shape

(121605, 128, 128, 1)

In [ ]:
###  NORMALIZE PATCHES 
folder_save = "/scratch/fslippe/modis/MOD02/training_data/patch_size_%s/" %(patch_size)
model_run_name = "dnb_l95_z50_ps%s_band29_%s" %(patch_size, str(dates[0])[:4])

np.save(folder_save + "train_" + model_run_name, patches)
np.save(folder_save + "test_" + model_run_name, val_data)


In [9]:
patches_per_file = 50000 

for i in range(0, len(patches), patches_per_file):
    print(i)
    chunk = patches[i: i+patches_per_file]
    write_tfrecord(f'/scratch/fslippe/
    /MOD02/training_data/tf_data/normalized_trainingpatches_{model_run_name}_{i//patches_per_file}.tfrecord', chunk)

np.save("/uio/hume/student-u37/fslippe/data/models/max_vals_%s.npy" %(model_run_name), max_val)
np.save("/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_valpatches_%s" %(model_run_name), val_data)

0
50000
100000
